In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    #!git clone https://github.com/mjwock/DeepFLaSH_Pytorch.git /content/drive/My\ Drive/DeepFLaSH_Pytorch/FastAI
    %cd /content/drive/My\ Drive/DeepFLaSH_Pytorch/Pytorch
    #!git pull
except:
    pass

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/DeepFLaSH_Pytorch/Pytorch


In [2]:
from deepflash import preproc, unetadaption, losses, mask_converter

from skimage import io
from tqdm import tqdm

!pip install tifffile
from tifffile import imsave

!pip install Pillow
from PIL import Image
import json
import random
import os
import shutil
from pathlib import Path, PurePath
import numpy as np
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt

Using TensorFlow backend.


     |████████████████████████████████| 133kB 4.2MB/s 
     |████████████████████████████████| 706kB 7.6MB/s 


#Settings

General Setting

In [0]:
DIR = '/content/drive/My Drive/DeepFLaSH_Pytorch/Pytorch/data'
WORKING_DIR = '/content/drive/My Drive/DeepFLaSH_Pytorch/FastAI2/data/temp_data'

TYPE = 'cFOS'   # 'cFOS','Parv','NeuN'
CODER = ['gt']  # 'gt','cora','corinna','dennis','manju','manual','rohini','unet_cv10','unet_cv36'
COLOR = 'red'   # 'red','green'

TILE_SHAPE = (540,540)
TARGET_SIZE = (365,365)    # padding depends on used architecture, use torchsummary to determine target size
EL_SIZE = [635.9, 635.9]

CALC_WEIGHTS = False      # calculate weights for lossfunction, else load saved weights
WEIGHT_FILE = 'weightmaps.npy'

CHANGEAXIS = False         # pixelwise mask representation instead of multimask
                           # if True shape is (w,h,2) instead of (2,w,h)

SAVE_SEPERATELY = True    # save masks and weights to seperate tiff files

SEED = 42

In [0]:
# ONLY CHANGE FOR DIFFERENT FOLDER STRUCTURE
# images: data -> images -> color
# masks:  data -> labels -> type -> coder
IMG_PATH = f'{DIR}/images/{COLOR}'
MASK_PATH = f'{DIR}/labels/{TYPE}'

Weight Settings

In [0]:
LAMBDA = 50 #50
V_BAL = 0.1 #0.1
SIGMA_BAL = 10 #10 
SIGMA_SEP = 6 #6

#Read Files

Load Masks

In [0]:
filenames = os.listdir(IMG_PATH)

img_ids = [os.path.splitext(files)[0] for files in filenames] #get list with img_ids

In [0]:
coder_masks = {}
mask_ext = None
for coder in CODER:
  temp_path = f'{MASK_PATH}/{coder}'

  if mask_ext == None:
    mask_ext = os.path.splitext(os.listdir(temp_path)[0])[1] #get mask extension

  try:
    mask_list = [io.imread(f'{temp_path}/{img_id}{mask_ext}', as_gray=True).astype('int') 
                for img_id in img_ids]
    coder_masks[coder] = mask_list
  except:
    print(f'Can\'t find mask for image with img_id = {img_id} \n {temp_path}')

In [0]:
for coder in CODER:
  values = np.unique(coder_masks[coder])
  assert len(values) == 2, f'This is not a binary mask ({coder})'
  assert 0 in values and 1 in values, f'Your mask should only countain 0 or 1 ({coder})'

Load or calculate weights

In [9]:
if CALC_WEIGHTS:
  img_tensors = [{'rawdata': img, 'element_size_um': EL_SIZE} 
                 for img in image_list]

  #create dictionary for each coder:
  coder_weights = {}
  for coder in CODER:
    mask_list = coder_masks[coder]
    pre = preproc.DataPreProcessor(element_size_um=None,
                                border_weight_sigma_px=SIGMA_SEP,
                                foreground_dist_sigma_px=SIGMA_BAL,
                                border_weight_factor=LAMBDA,
                                foreground_background_ratio=V_BAL
                                )
    weight_list = []
    for i in tqdm(range(len(img_tensors))):
      sampleData, sampleLabels, sampleWeights, samplePdf = pre.generateSample(img_tensors[i], classlabels=mask_list[i])
      weight_list.append(sampleWeights)  

    coder_weights[coder] = weight_list

  #save weights
  np.save(f'{DIR}/{WEIGHT_FILE}',coder_weights)

else:
  #load saved weights
  coder_weights = np.load(f'{DIR}/{WEIGHT_FILE}',allow_pickle=True)[()] # [()] turns loaded array to dict
  print(f'Weights loaded (\'{WEIGHT_FILE}\')')

Weights loaded ('weightmaps.npy')


Stacking masks

In [10]:
assert len(CODER)==1, print('This function only works for single coder')
coder = 'gt'
masks = [np.asarray([mask_list[i],coder_weights[coder][i]]) 
          for i in range(len(img_ids))]
np.shape(masks)

(36, 2, 1024, 1024)

Changing axis

In [15]:
if CHANGEAXIS:
  for i, mask in enumerate(masks):
    masks[i] = np.moveaxis(mask, 0, -1)
np.shape(masks)

(36, 2, 1024, 1024)

Create Working Directory

In [0]:
if os.path.exists(WORKING_DIR):
  shutil.rmtree(WORKING_DIR)

os.makedirs(f'{WORKING_DIR}/labels')

if SAVE_SEPERATELY:
  os.makedirs(f'{WORKING_DIR}/weights')

Save masks

In [31]:
for index, id in enumerate(img_ids):
  if SAVE_SEPERATELY:
    imsave(f'{WORKING_DIR}/labels/{id}.tif',np.array(masks[index][0]))
    imsave(f'{WORKING_DIR}/weights/{id}.tif',np.array(masks[index][1]))
    print(f'Saved label and corresponding weight {id}.tif')
  else:
    imsave(f'{WORKING_DIR}/labels/{id}.tif',np.array(masks[index]))
    print(f'Saved label {id}.tif')

Saved label and corresponding weight 0001.tif
Saved label and corresponding weight 0008.tif
Saved label and corresponding weight 0062.tif
Saved label and corresponding weight 0063.tif
Saved label and corresponding weight 0067.tif
Saved label and corresponding weight 0015.tif
Saved label and corresponding weight 0011.tif
Saved label and corresponding weight 0064.tif
Saved label and corresponding weight 0004.tif
Saved label and corresponding weight 0007.tif
Saved label and corresponding weight 0006.tif
Saved label and corresponding weight 0068.tif
Saved label and corresponding weight 0072.tif
Saved label and corresponding weight 0078.tif
Saved label and corresponding weight 0069.tif
Saved label and corresponding weight 0074.tif
Saved label and corresponding weight 0089.tif
Saved label and corresponding weight 0077.tif
Saved label and corresponding weight 0091.tif
Saved label and corresponding weight 0082.tif
Saved label and corresponding weight 0076.tif
Saved label and corresponding weig

Test saved masks

In [0]:
TEST_ID = '0015'
test_mask = io.imread(f'{WORKING_DIR}/labels/{TEST_ID}.tif')

test_weights = io.imread(f'{WORKING_DIR}/weights/{TEST_ID}.tif')

In [28]:
print(f'Shape: {np.shape(test_mask)}')
print(test_mask)

print(f'Shape: {np.shape(test_weights)}')
print(test_weights)

Shape: (1024, 1024)
[[0. 0. 0. 0. ... 0. 0. 0. 0.]
 [0. 0. 0. 0. ... 0. 0. 0. 0.]
 [0. 0. 0. 0. ... 0. 0. 0. 0.]
 [0. 0. 0. 0. ... 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. ... 0. 0. 0. 0.]
 [0. 0. 0. 0. ... 0. 0. 0. 0.]
 [0. 0. 0. 0. ... 0. 0. 0. 0.]
 [0. 0. 0. 0. ... 0. 0. 0. 0.]]
Shape: (1024, 1024)
[[0.1 0.1 0.1 0.1 ... 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 ... 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 ... 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 ... 0.1 0.1 0.1 0.1]
 ...
 [0.1 0.1 0.1 0.1 ... 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 ... 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 ... 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 ... 0.1 0.1 0.1 0.1]]


Convert masks to (0...N) for N classes

In [0]:
#mask_converter.convertMasksToGrayscaleZeroToN(f'{MASK_PATH}/{CODER[0]}', f'/content/drive/My Drive/DeepFLaSH_Pytorch/FastAI/data/converted_labels/{CODER[0]}')

array([0., 1.])